In [1]:
import requests
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
from glob import glob
import time
import json
import os
import gc

In [2]:
files = glob('topics/*.json')

topics = []
for f in files:
    with open(f) as fopen:
        topics.extend(json.load(fopen))
        
topics = list(set(topics))
topics = sorted(topics)

In [3]:
len(topics)

375073

In [4]:
import time

def get_href_topic(url, topic = None):
    while True:
        try:
            r = requests.get(url)
            break
        except:
            time.sleep(1.0)
    soup = BeautifulSoup(r.content, "lxml")
    a = soup.find_all('a')
    a = [a_.get('href') for a_ in a if a_.get('href')]
    if topic:
        try:
            max_page = max([int(a_.split('page-')[-1]) for a_ in a if topic in a_ and '/page-' in a_])
        except:
            max_page = -1
    else:
        max_page = -1
    div = soup.find_all('div', {'class': 'message-userContent lbContainer js-lbContainer'})
    return url, [str(d) for d in div], max_page


t = topics[3]
url = f'https://carigold.com{t}'
get_href_topic(url, t)

('https://carigold.com/forum/threads/%C2%A8%C2%A8expectations.31918/',
 ['<div class="message-userContent lbContainer js-lbContainer" data-lb-caption-desc="ayitin69 · Mar 10, 2008 at 7:36 PM" data-lb-id="post-1082213">\n<article class="message-body js-selectToQuote">\n<div>\n<div class="bbWrapper">The week has started off quietly enough although Asia did see some mild selling pressure for the Greenback. However, with a relatively subdued release calendar this week the willingness to probe new lows has not been present. <br/>\n<br/>\nFor the States Thursday’s advance retail sales are probably the most influential but still will unlikely manage to generate any significant move in the Dollar. From Europe there is a strong of industrial production and inflation numbers that should maintain the recent positive bounce that has been seen this year.<br/>\n<br/>\nIt seems unlikely that economic releases are going to generate any additional sales in the Dollar and we are therefore left to watch 

In [5]:
# !rm -rf subtopics

In [8]:
!mkdir subtopics

mkdir: cannot create directory ‘subtopics’: File exists


In [9]:
max_worker = 50
for i in tqdm(range(0, len(topics), max_worker)):
    
    filename = os.path.join('subtopics', f'{i}.json')
    if os.path.exists(filename):
        continue
    
    gc.collect()
    
    urls = [(f'https://carigold.com{t}', t) for t in topics[i: i + max_worker]]
    results = []
    with ThreadPoolExecutor(max_workers=max_worker) as executor:
        futures = {executor.submit(get_href_topic, url[0], url[1]): url for url in urls}

        for future in as_completed(futures):
            results.append(future.result())
    
    with open(filename, 'w') as fopen:
        json.dump(results, fopen)
        
    try:
        del results
        del futures
    except:
        pass

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████| 7502/7502 [00:00<00:00, 564568.64it/s]


In [10]:
!du -hs subtopics

2.9G	subtopics


In [11]:
# with open()

In [12]:
# for i in tqdm(range(len(a))):
#     t = topics[i]
#     filename = os.path.join('topics', f'{i}.json')
#     if os.path.exists(filename):
#         continue
        
#     base_url = f'https://carigold.com{t}'
    
#     filtered, max_page = get_href(base_url)
    
#     for i in tqdm(range(2, max_page + 1, max_worker)):
#         aranged = np.arange(i, i + max_worker)
#         urls = [f'{base_url}page-{a}' for a in aranged]

#         with ThreadPoolExecutor(max_workers=max_worker) as executor:
#             futures = {executor.submit(get_href, url): url for url in urls}

#             for future in as_completed(futures):
#                 filtered.extend(future.result()[0])
                
#     topics = list(set(filtered))
#     with open(filename, 'w') as fopen:
#         json.dump(topics, fopen)